In [1]:
import pandas as pd
import numpy as np
import os

## En datos/properati deben estar todos los archivos de ventas provistos por properati

In [2]:
filesProperati = filter( lambda f: not f.startswith('.'), os.listdir("datos/properati"))

In [3]:

dataframes = []
for file_name in filesProperati:
    
    newDataFrame = pd.read_csv("datos/properati/"+file_name)
    dataframes.append(newDataFrame)

       

## ¿Como filtramos por las zonas pedidas?

In [4]:
nulosStateName = 0
nulosPlaceWParentNames = 0
noCuentaConStateName = 0
noCuentaConPlaceWParentNames = 0
for df in dataframes:
    try:
        nulosStateName += df.state_name.isnull().sum()
    except:
        noCuentaConStateName += 1
    try:
        nulosPlaceWParentNames += df.place_with_parent_names.isnull().sum()
    except:
        noCuentaConPlaceWParentNames += 1
print "Cantidad de registros con valor nulo en state_name: "+str(nulosStateName)
print "Cantidad de registros con valor nulo en place_w_parent_names: "+str(nulosPlaceWParentNames)
print "Cantidad de archivos que no cuentan con la columna state_name: "+str(noCuentaConStateName)
print "Cantidad de archivos que no cuentan con la columna place_w_parent_names: "+str(noCuentaConPlaceWParentNames)

Cantidad de registros con valor nulo en state_name: 0
Cantidad de registros con valor nulo en place_w_parent_names: 0
Cantidad de archivos que no cuentan con la columna state_name: 26
Cantidad de archivos que no cuentan con la columna place_w_parent_names: 0


## Dado los resultados, para quedarnos solo con las propiedades de Capital Federal y GBA tendremos que utilizar la columna 'place_with_parent_names'

In [5]:
frecuencias = {}
for df in dataframes:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [6]:
for zona in frecuencias: 
    print zona,frecuencias[zona]

San Juan 568
Córdoba 99129
Chaco 1868
Mendoza 8315
Colonia 205
Chubut 1194
Capital Federal 656229
San Luis 6929
Tucumán 3968
Misiones 11127
Tierra Del Fuego 890
Bs.As. G.B.A. Zona Oeste 304893
Corrientes 1436
Formosa 341
Maldonado 6101
Río Negro 17661
Santa Cruz 242
New York 41
California 29
Montevideo 3281
Neuquén 11194
La Pampa 5134
Buenos Aires Costa Atlántica 301094
Buenos Aires Interior 62686
Miami 987
Florida 1187
Bs.As. G.B.A. Zona Sur 167017
Santa Catarina 268
Santiago Del Estero 282
Rocha 85
Jujuy 502
Punta del Este 15209
Santa Fe 117984
Salta 7621
La Rioja 333
Entre Ríos 16068
Bs.As. G.B.A. Zona Norte 541028
Catamarca 886


## Interesan: Bs.As. G.B.A. Zona Norte, Bs.As. G.B.A. Zona Sur, Bs.As. G.B.A. Zona Oeste, Capital Federal


In [7]:
def filtrarCapitalYGBA(serie):
    deseados = ["Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Oeste", "Capital Federal"]
    booleans = []
    for item in serie:
        if item.split("|")[2] in deseados:
            booleans.append(True)
        else:
            booleans.append(False)
    return booleans

In [8]:
filtrados =[]
for df in dataframes:
    
    df = df[filtrarCapitalYGBA(df['place_with_parent_names'])]
    filtrados.append(df)

### Por las dudas chequeo que ahora las frecuencias sean las mismas y solo queden las zonas que interesan

In [9]:
frecuencias = {}
for df in filtrados:
    parentNames = df['place_with_parent_names']
    for padres in parentNames:
        zona = padres.split("|")[2]
        if zona not in frecuencias:
            frecuencias[zona] = 1
        else:
            frecuencias[zona]+=1

In [10]:
for key in frecuencias:
    print key, frecuencias[key]

Capital Federal 656229
Bs.As. G.B.A. Zona Norte 541028
Bs.As. G.B.A. Zona Oeste 304893
Bs.As. G.B.A. Zona Sur 167017


# Ahora recupero datos, elimino algunas columnas, y demas.
# Primero miro como estan los datos, para ver si conviene trabajar con los precios en pesos argentinos o dolares. 

In [11]:
def a(df, column_name):
    try:
        return(df[column_name].isnull().sum(),0)
    except:
        return (0,1)

In [12]:
nulsPriceAproxLocalCurr = 0
nulsPriceAproxUsd = 0
nulsPricePerM2 = 0
nulsPricePerM2usd = 0

priceAproxLocalCurrInex = 0
priceAproxUsdInex = 0
pricePerM2Inex = 0
pricePerM2usdInex = 0

for df in filtrados:
    nuls, inex = a(df, "price_aprox_local_currency")
    nulsPriceAproxLocalCurr += nuls
    priceAproxLocalCurrInex += inex
    nuls,inex = a(df, "price_aprox_usd")
    nulsPriceAproxUsd += nuls
    priceAproxUsdInex += inex
    nuls, inex = a(df, "price_per_m2")
    nulsPricePerM2 += nuls
    pricePerM2Inex += inex
    nuls,inex = a(df, "price_usd_per_m2")
    nulsPricePerM2usd += nuls
    pricePerM2usdInex += inex

In [13]:
print str(nulsPriceAproxLocalCurr)+" Nulls priceAproxLocalCurr"
print str(nulsPriceAproxUsd)+" Nulls priceAproxUsd"
print str(nulsPricePerM2)+" Nulls Price Per M2"
print str(nulsPricePerM2usd)+" Nulls pricePerM2USD"
print str(priceAproxLocalCurrInex)+" Inexistentes priceAproxLocalCurr"
print str(priceAproxUsdInex)+" Inexistentes priceAproxUsd"
print str(pricePerM2Inex)+" Inexistentes pricePerM2"
print str(pricePerM2usdInex)+" Inexistentes pricePerM2usd"

213656 Nulls priceAproxLocalCurr
213656 Nulls priceAproxUsd
93534 Nulls Price Per M2
639946 Nulls pricePerM2USD
0 Inexistentes priceAproxLocalCurr
0 Inexistentes priceAproxUsd
26 Inexistentes pricePerM2
0 Inexistentes pricePerM2usd


# Tambien miro los datos acerca de las superficies

## Observo que algunos archivos solo tienen una columna 'surface_in_m2', no distingue entre si es cubierta o si es total, suponemos que es la superficie total. 

In [14]:
nulsSuperficieTotal = 0
for df in filtrados:
    try:
        nulsSuperficieTotal += df.surface_in_m2.isnull().sum()
    except:
        nulsSuperficieTotal += df.surface_total_in_m2.isnull().sum()
nulsSuperficieTotal

449915

In [15]:
#Cuenta cuantas propiedades tienen el precio por m2 en dolares nulo, 
#pero el precio local no nulo
count = 0
for df in filtrados:
    try:
        count += len(df[~df.price_per_m2.isnull() & df.price_usd_per_m2.isnull()])
    except:
        #Algunos dataframes no tienen el precio por m2 en pesos
        continue
print "Cantidad de propiedades de las que se conoce el precio por m2 (En pesos)\
 pero no en dolares: "+str(count)

Cantidad de propiedades de las que se conoce el precio por m2 (En pesos) pero no en dolares: 91723


# ¿La columna price, tiene el mismo precio que price_aprox_usd o el mismo precio que price_aprox_local_curr? solo analizamos para el caso en el que no este el precio en dolares

In [16]:
mismo_usd = 0
mismo_aprox_local_curr = 0
for df in filtrados:
    mismo_aprox_local_curr += len(df[(df["price_aprox_local_currency"] == df["price"]) &\
                                    (df["price"] != df["price_aprox_usd"]) & \
                                    ~df["price_aprox_usd"].isnull()])
    mismo_usd += len(df[df["price_aprox_usd"] == df["price"]])

print mismo_aprox_local_curr
print mismo_usd
    

1
1321544


# Salvo un caso, la columna price contiene los mismos valores que la columna price_aprox_usd. Descarto la columna price. ¿Puedo recuperar precio de propiedades en dolares a partir del precio aproximado local?

In [17]:
coincidencias = 0
for df in filtrados:
    coincidencias += len(df[(~df["price_aprox_local_currency"].isnull() & df["price_aprox_usd"].isnull()) |\
                            (~df["price"].isnull() & df["price_aprox_usd"].isnull())])

print coincidencias
    

2


# Son tan solo dos casos en los que el precio aproximado local no es nulo y el precio en dolares lo es. No tenerlo en cuenta no deberia afectarnos

# Se pueden recuperar datos de los precios en dolares por m2 realizando la correspondiente conversion. De igual manera, antes estudio para ver que no se pueda recuperar de otra manera. ¿A cuantos se les desconoce el precio por m2 en USD y no se conoce el precio total (en USD) o la superficie total?

In [18]:
coincidencias = 0
for df in filtrados:
    if "price_per_m2" in df.columns:
        coincidencias += len(df[(~df["price_per_m2"].isnull() & df["price_usd_per_m2"].isnull())\
                           & (df["price_aprox_usd"].isnull() | df["surface_total_in_m2"].isnull())])

print coincidencias    

78325


# 78325 de 639946 registros pueden ser recuperados si realizamos la conversion 

# Los datos para las conversiones fueron encontrados en:
### www.bcra.gob.ar/Pdfs/PublicacionesEstadisticas/com3500.xls
# Los mismos fueron formateados de manera conveniente por como se nos presentan los datos de properati, usando LibreOffice Calc

In [19]:

datosDolar = pd.read_csv("info_dolar.csv",index_col = "Fecha")

In [20]:
from datetime import datetime
from datetime import timedelta
def recuperar_precio_m2_en_dolares(rowProperati):
    if ~pd.isnull(rowProperati["price_per_m2"]) & pd.isnull(rowProperati["price_usd_per_m2"]):
        fechaCreacionPropiedad = rowProperati["created_on"]
        anho = int(fechaCreacionPropiedad.split("-")[0])
        mes = int(fechaCreacionPropiedad.split("-")[1])
        dia = int(fechaCreacionPropiedad.split("-")[2])
        fechaConCotizacion = datetime(anho, mes, dia)
        fechaEncontrada = False
        while not(fechaEncontrada):
            try:
                cotizFechaCreacProp = float(datosDolar.ix[fechaConCotizacion.strftime('%Y-%m-%d')]["Cotizacion Dolar"])
                fechaEncontrada = True
            except:
                #En esa fecha no se habia publicado informacion del dolar (no vario)
                #Me fijo en el dia anterior
                fechaConCotizacion = fechaConCotizacion - timedelta(days = 1)
        return cotizFechaCreacProp*rowProperati["price_per_m2"]
    else:
        return rowProperati["price_per_m2"]

In [ ]:
for df in filtrados:
    if "price_per_m2" in df.columns:
        df["price_usd_per_m2"] = df.apply(lambda row: recuperar_precio_m2_en_dolares(row),axis=1)

# Corroboro

In [22]:
coincidencias = 0
for df in filtrados:
    if "price_per_m2" in df.columns:
        coincidencias += len(df[(~df["price_per_m2"].isnull() & df["price_usd_per_m2"].isnull())\
                           & (df["price_aprox_usd"].isnull() | df["surface_total_in_m2"].isnull())])

print coincidencias    

0


# Se recuperaron 78325 registros mediante la conversion a dolares del precio del metro cuadrado

# ¿Se puede recuperar mas informacion?

In [ ]:
#Primero le pongo el mismo nombre a la superficie total
for df in filtrados:
    try:
        df['surface_total_in_m2']
    except:
        df.rename(columns={'surface_in_m2':'surface_total_in_m2'},inplace=True)
    

In [24]:
#S 'se tiene informacion de' N 'no se tiene informacion de'
S_ppm2_S_sup_N_precio = 0
S_ppm2_S_precio_N_sup = 0
S_sup_S_precio_N_ppm2 = 0
for df in filtrados:
    S_ppm2_S_sup_N_precio += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["surface_total_in_m2"].isnull() &\
                                    df["price_aprox_usd"].isnull()])
    S_ppm2_S_precio_N_sup += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["surface_total_in_m2"].isnull()])
    S_sup_S_precio_N_ppm2 += len(df[~df["surface_total_in_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["price_usd_per_m2"].isnull()])
print S_ppm2_S_sup_N_precio
print S_ppm2_S_precio_N_sup
print S_sup_S_precio_N_ppm2

278
78325
52089


# Recupero esta informacion

In [25]:
from utils_recuperacion_datos import recuperar_superficie,recuperar_precio_usd,recuperar_ppm2

In [ ]:
for df in filtrados:
     df['surface_total_in_m2'] = df.apply(lambda row: recuperar_superficie(row),axis=1)
     df['price_aprox_usd'] = df.apply(lambda row: recuperar_precio_usd(row),axis=1)
     df['price_usd_per_m2'] = df.apply(lambda row: recuperar_ppm2(row),axis=1)

## Chequeo que se hayan recuperado datos

In [27]:
#S 'se tiene informacion de' N 'no se tiene informacion de'
S_ppm2_S_sup_N_precio = 0
S_ppm2_S_precio_N_sup = 0
S_sup_S_precio_N_ppm2 = 0
for df in filtrados:
    S_ppm2_S_sup_N_precio += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["surface_total_in_m2"].isnull() &\
                                    df["price_aprox_usd"].isnull()])
    S_ppm2_S_precio_N_sup += len(df[~df["price_usd_per_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["surface_total_in_m2"].isnull()])
    S_sup_S_precio_N_ppm2 += len(df[~df["surface_total_in_m2"].isnull() &\
                                    ~df["price_aprox_usd"].isnull() &\
                                    df["price_usd_per_m2"].isnull()])
print S_ppm2_S_sup_N_precio
print S_ppm2_S_precio_N_sup
print S_sup_S_precio_N_ppm2

0
0
0


## A continuacion unifico y voy eliminando duplicados, teniendo en cuenta distintos subgrupos de columnas

In [28]:
unificacion = pd.concat(filtrados, axis=0, ignore_index=True)

In [31]:
len(unificacion)

1669167

In [32]:
#Elimino los registros que tengan todas las columnas identicas
a = unificacion.drop_duplicates()

In [33]:
len(a)

1339383

In [34]:
#Los que no tienen ni precio aproximado en dolares ni precio por m2 en dolar ya no me interesan
b = a[~a["price_aprox_usd"].isnull() | ~a["price_usd_per_m2"].isnull()]

In [35]:
len(b)

1205487

In [36]:
links_repetidos = b["properati_url"].value_counts()

# Observo un poco que informacion encuentro en estas publicaciones con links repetidos

In [37]:
links_repetidos.head(2)

http://www.properati.com.ar/95fn_venta_departamento_capital-federal                                       22
http://www.properati.com.ar/7v2b_venta_departamento_olivos_avenida-del-libertador-gral-san-martin_2300    22
Name: properati_url, dtype: int64

In [38]:
b[b["properati_url"] == "http://www.properati.com.ar/95fn_venta_departamento_capital-federal"].head(5)

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_aprox_usd,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,title
74702,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,http://cf-thumbs.properati.com.ar/IutzRsfg9NDG...,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
137024,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,http://cf-thumbs.properati.com.ar/IutzRsfg9NDG...,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
207175,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,http://cf-thumbs.properati.com.ar/c7mTrSMGO5ke...,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
236474,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,http://cf-thumbs.properati.com.ar/CIuogg3sb46t...,...,117900.0,NaN,1871.428571,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,63.0,NaN
267336,NaN,2013-08-12,USD,NaN,NaN,NaN,11.0,3433955.0,NaN,http://d1t4y5hzjn9jnl.cloudfront.net/siM2ypcMK...,...,117900.0,NaN,NaN,http://www.properati.com.ar/95fn_venta_departa...,apartment,1.0,NaN,NaN,NaN,NaN


In [39]:
b[b["properati_url"] == "http://www.properati.com.ar/7v2b_venta_departamento_olivos_avenida-del-libertador-gral-san-martin_2300"].head(5)

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_aprox_usd,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_total_in_m2,title
69454,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,http://cf-thumbs.properati.com.ar/OG0XIZ_02IJD...,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
130136,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,http://cf-thumbs.properati.com.ar/OG0XIZ_02IJD...,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
195104,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,http://cf-thumbs.properati.com.ar/tvdlAIu9J6BP...,...,185000.0,NaN,3936.170213,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
227790,NaN,2013-06-14,USD,NaN,NaN,NaN,NaN,3430310.0,NaN,http://cf-thumbs.properati.com.ar/llZM66nzNfOV...,...,192000.0,NaN,4085.106383,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,47.0,NaN
265791,NaN,2013-06-14,USD,NaN,NaN,NaN,6.0,3430310.0,NaN,http://d1t4y5hzjn9jnl.cloudfront.net/0AdijljJB...,...,195000.0,NaN,NaN,http://www.properati.com.ar/7v2b_venta_departa...,apartment,2.0,NaN,NaN,NaN,NaN


# Observo que hay publicaciones con misma URL y que parecen ser las mismas propiedades, pero el inconveniente es que los campos vacios de cada uno de estos registros varia, debo filtrar de una forma mas compleja

# Antes de hacer esto voy a eliminar columnas que no interesan

In [40]:
len(b[~b["lat"].isnull() & ~b["lon"].isnull() & b["lat-lon"].isnull()])

0

In [41]:
#Puedo eliminar latitud y longitud, ya que lat-lon tiene su informacion
#tambien, country_name, image_thumbnail, operation, price, price aprox local curr, currency
#ya que solo trabajaremos con precios en dolares


In [42]:
b.columns

Index([u'country_name', u'created_on', u'currency', u'description',
       u'expenses', u'extra', u'floor', u'geonames_id', u'id',
       u'image_thumbnail', u'lat', u'lat-lon', u'lon', u'operation',
       u'place_name', u'place_with_parent_names', u'price',
       u'price_aprox_local_currency', u'price_aprox_usd', u'price_per_m2',
       u'price_usd_per_m2', u'properati_url', u'property_type', u'rooms',
       u'state_name', u'surface_covered_in_m2', u'surface_total_in_m2',
       u'title'],
      dtype='object')

In [43]:
data = b.drop(["country_name","lat","lon","country_name","image_thumbnail",\
               "operation","price",'price_per_m2',"price_aprox_local_currency","currency"],axis=1)

In [44]:
#Elimino registros con misma url de properati, mismo precio y misma cantidad de habitaciones y piso si son casas
casas = data[data["property_type"] == "house"]


In [45]:
len(casas)

390251

In [46]:
casas_duplicadas = casas[casas.duplicated(subset = ["properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor"])]

In [47]:
len(casas_duplicadas)

218206

In [48]:
casas_unicas = casas.drop_duplicates(subset = ["properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor"])

In [49]:
resto_propiedades = data[data["property_type"] != "house"]

In [50]:
print len(casas_unicas)+len(resto_propiedades)

987281


In [51]:
data_juntada = pd.concat([resto_propiedades,casas_unicas])

In [52]:
len(data_juntada)

987281

In [53]:
stores = data_juntada[data_juntada["property_type"] == "store"]

In [54]:
stores_duplicadas = stores[stores.duplicated(subset=["properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor"])]

In [55]:
print len(stores)

19748


In [56]:
len(stores_duplicadas)

8900

In [57]:
stores_duplicadas["properati_url"].value_counts().head(5)

http://www.properati.com.ar/dya4_venta_local_capital-federal                       8
http://www.properati.com.ar/e3q3_venta_local_san-martin                            8
http://www.properati.com.ar/g62v_venta_local_wilde                                 8
http://www.properati.com.ar/i6d3_venta_local_lanus-este_avenida-9-de-julio_1300    8
http://www.properati.com.ar/hvs7_venta_local_general-rodriguez                     8
Name: properati_url, dtype: int64

In [58]:
stores_unicos = stores.drop_duplicates(subset=["properati_url","price_aprox_usd","price_usd_per_m2","rooms","floor"])

In [59]:
len(stores_unicos)

10848

In [60]:
resto_props = data_juntada[data_juntada["property_type"] != "store"]

In [61]:
data = pd.concat([resto_props,stores_unicos])

In [63]:
sin_publicaciones_duplicadas = data[(~data.duplicated()) | (data['properati_url'].isnull())]


In [64]:
len(sin_publicaciones_duplicadas)

686484

In [65]:
sin_publicaciones_duplicadas.to_csv("datosCabaYGBA.csv")